 # Optimal solver for 2x2 Rubik’s cube

Theoretically there should be $\frac{8!\times 3^7}{24}$ different configurations for the 2x2x2 cube. 8! comes from the fact that there are 8 cubies that can be arranged in any order. If we fix one of the eight cubies then in relation to that cubie, each other cubie can have different orientation of its colors. So $3^7$ comes from 3 different independant orientations for 7 non-fixed cubies.<br>
The resulting number is divided by 24, because 2x2x2 has no inherent orientation and can therefore have multiple solutions among previously found search space. 


Goal: 
 * Find number of different configurations of 2x2x2 Rubik's cube 
 * Find God's number
 
Strategy:
 * Construct a solved cube
 * Using legal moves generate all possible states that are obtainable from solved cube.
 * Save the minimum distance to any given state.
 * If no new states are after a round of BFS, then stop.
   * God's number is maximum distance from solved cube


## Building the solved cube and defining moves

In [1]:
RED = 2
GREEN = 1
WHITE = 0
ORANGE=3
BLUE = 4
YELLOW = 5

In [2]:
import numpy
class Face2x2: 
    
    def sum_colors(self):
        return self.UR**4+self.UL**4+self.LL**4+self.LR**4
    
    def __init__(self, UL, UR, LL, LR):
        self.UL = UL
        self.UR = UR
        self.LL = LL
        self.LR = LR
    
    def  __str__(self):
        return str([self.UL, self.UR,self.LL, self.LR])
    
    def __repr__(self):
        return str(1*self.UL + 6*self.UR + 36*self.LL +  216 * self.LR)
    
    def __hash__(self):
        a = 1*self.UL + 6*self.UR + 36*self.LL +  216 * self.LR
        return a
    
    def __eq__(self, other):
        return hash(self)==hash(other)
    
class Cube:
    
    left = Face2x2(BLUE, BLUE, BLUE, BLUE)
    right = Face2x2(GREEN, GREEN, GREEN, GREEN)
    top = Face2x2(YELLOW, YELLOW, YELLOW, YELLOW)
    bottom = Face2x2(WHITE, WHITE, WHITE, WHITE)
    face =  Face2x2(RED, RED, RED, RED)
    back = Face2x2(ORANGE, ORANGE, ORANGE, ORANGE)
    
    

    def get_opposite(self, side):
        if side==self.left:
            return self.right
        if side==self.right:
            return self.left
        if side==self.bottom:
            return self.top
        if side==self.top:
            return self.bottom
        if side==self.face:
            return self.back
        if side==self.back:
            return self.face
    
    def get_invariants(self):
        invs = set()
        a = self.lcw().rccw()
        b = self.lcw().rccw().lcw().rccw()
        c = self.rcw().lccw()
        d = self
        e = self.tcw().bccw()
        f = self.bcw().tccw() 
        
        for side in [a,b,c,d,e,f]:
            invs.add(side)
            invs.add(side.Bcw().Fccw())
            invs.add(side.Fcw().Bccw())
            invs.add(side.Bcw().Fccw().Bcw().Fccw())
            assert(side.Bcw().Fccw().Bcw().Fccw() == side.Fcw().Bccw().Fcw().Bccw())
            assert(side.Fcw().Bccw().Fcw().Bccw().Fcw().Bccw() == side.Bcw().Fccw())

        return invs
        
        
    def get_neighs(self, side):
        if side==left:
            return [top, bottom, face, back]
        if side==right:
            return [top, bottom, back, face]
        if side==bottom:
            return [face, back, right, left]
        if side==top:
            return [back, face, right, left]
        if side==face:
            return [top, bottom, right, left]
        if side==back:
            return [top, bottom, left, right]
        
    def is_on_right_side(self, a, b, side):
        if (a==self.face):
            if b in [self.top, self.right]:
                return side in [self.right, self.bottom]
            return side in [self.left, self.top]
        
        if (a==self.back):
            if b in [self.top, self.left]:
                return side in [self.left, self.bottom]
            return side in [self.right, self.top]
        
        if (a==self.left):
            if b in [self.top, self.face]:
                return side in [self.face, self.bottom]
            return side in [self.top, self.back]
        
        if (a==self.right):
            if b in [self.top, self.back]:
                return side in [self.back, self.bottom]
            return side in [self.face, self.top]
        
        if (a==self.top):
            if b in [self.back, self.right]:
                return side in [self.face, self.right]
            return side in [self.back, self.left]
        
        if (a==self.bottom):
            if b in [self.face, self.right]:
                return side in [self.right, self.back]
            return side in [self.face, self.left]
        

    # Compute heavy method
    def reorient(self):
        new_face = max([self.left, self.right,self.bottom,self.top,self.face, self.back], key= lambda x: x.sum_colors())

        if new_face == self.left:
            self = self.bcw().tccw()
        elif new_face == self.right:
            self= self.tcw().bccw()
        elif new_face == self.bottom:
            self= self.rcw().lccw()
        elif new_face == self.top :
            self= self.lcw().rccw()
        elif new_face == self.back:
            self= self.tcw().bccw().tcw().bccw()
  
        new_top = max([self.left, self.right,self.bottom,self.top], key= lambda x: x.sum_colors())   
        
        if self == self.left:
            cube = self.Fcw().Bccw()
        elif new_top == self.right:
            self= self.Bcw().Fccw()
        elif new_top == self.bottom:
            self= self.Bcw().Fccw().Bcw().Fccw()
    
        return self
    


    
    def lccw(self):
    
        l =Face2x2(self.left.UR, self.left.LR, self.left.UL, self.left.LL)
        btm=Face2x2(self.back.LL, self.bottom.UR, self.back.UL,self.bottom.LR)
        bc=Face2x2(self.top.LL, self.back.UR, self.top.UL,self.back.LR)
        t=Face2x2(self.face.UL, self.top.UR, self.face.LL,self.top.LR)
        f=Face2x2(self.bottom.UL, self.face.UR, self.bottom.LL,self.face.LR)
        r = Face2x2(self.right.UL, self.right.UR, self.right.LL,self.right.LR)
        
        return Cube(f, l, t, r, bc, btm)
    
    def lcw(self):
        
        btm=Face2x2(self.face.UL, self.bottom.UR, self.face.LL,self.bottom.LR)
        bc=Face2x2(self.bottom.LL, self.back.UR, self.bottom.UL,self.back.LR)
        t=Face2x2(self.back.LL, self.top.UR, self.back.UL,self.top.LR)
        f=Face2x2(self.top.UL, self.face.UR, self.top.LL,self.face.LR)
        r = Face2x2(self.right.UL, self.right.UR, self.right.LL,self.right.LR)
        l=Face2x2(self.left.LL, self.left.UL,self.left.LR,self.left.UR)

        return Cube(f, l, t, r, bc, btm)

    
    def rcw(self):
        l=Face2x2(self.left.UL, self.left.UR, self.left.LL, self.left.LR)
        r=Face2x2(self.right.LL, self.right.UL,self.right.LR,self.right.UR)
        btm=Face2x2(self.bottom.UL, self.back.LR, self.bottom.LL, self.back.UR)
        bc=Face2x2(self.back.UL, self.top.LR, self.back.LL, self.top.UR)
        t=Face2x2(self.top.UL, self.face.UR, self.top.LL, self.face.LR)
        f=Face2x2(self.face.UL, self.bottom.UR, self.face.LL, self.bottom.LR)
        
        return Cube(f, l, t, r, bc, btm)

    
    
    def rccw(self):
        l=Face2x2(self.left.UL, self.left.UR, self.left.LL, self.left.LR)
        r=Face2x2(self.right.UR, self.right.LR,self.right.UL,self.right.LL)
        t=Face2x2(self.top.UL, self.back.LR, self.top.LL, self.back.UR)
        bc=Face2x2(self.back.UL, self.bottom.LR, self.back.LL, self.bottom.UR)
        btm=Face2x2(self.bottom.UL, self.face.UR, self.bottom.LL, self.face.LR)
        f=Face2x2(self.face.UL, self.top.UR, self.face.LL, self.top.LR)
        
        return Cube(f, l, t, r, bc, btm)    

        
        
    def tcw(self):
        btm=Face2x2(self.bottom.UL, self.bottom.UR, self.bottom.LL, self.bottom.LR)
        t=Face2x2(self.top.LL, self.top. UL,self.top.LR, self.top.UR)
        f=Face2x2(self.right.UL, self.right.UR, self.face.LL, self.face.LR)
        r=Face2x2(self.back.UR, self.back.UL, self.right.LL, self.right.LR)
        bc=Face2x2(self.left.UR, self.left.UL, self.back.LL, self.back.LR)
        l=Face2x2(self.face.UL, self.face.UR, self.left.LL, self.left.LR)
        return Cube(f, l, t, r, bc, btm)


    def tccw(self):
        btm=Face2x2(self.bottom.UL, self.bottom.UR, self.bottom.LL, self.bottom.LR)
        t=Face2x2(self.top.UR, self.top.LR,self.top.UL,self.top.LL)
        f=Face2x2(self.left.UL, self.left.UR, self.face.LL, self.face.LR)
        l=Face2x2(self.back.UR, self.back.UL, self.left.LL, self.left.LR)
        bc=Face2x2(self.right.UR, self.right.UL, self.back.LL, self.back.LR)
        r=Face2x2(self.face.UL, self.face.UR, self.right.LL, self.right.LR)
        
        return Cube(f, l, t, r, bc, btm)
        
    def bcw(self):
        t=Face2x2(self.top.UL, self.top.UR, self.top.LL, self.top.LR)
        
        btm=Face2x2(self.bottom.LL, self.bottom.UL,self.bottom.LR,self.bottom.UR)
        f=Face2x2(self.face.UL, self.face.UR, self.left.LL, self.left.LR)
        l=Face2x2(self.left.UL, self.left.UR, self.back.LR, self.back.LL)
        bc=Face2x2(self.back.UL, self.back.UR, self.right.LR, self.right.LL)
        r=Face2x2(self.right.UL, self.right.UR, self.face.LL, self.face.LR)
        
        return Cube(f, l, t, r, bc, btm)
        
    def bccw(self):
        t=Face2x2(self.top.UL, self.top.UR, self.top.LL, self.top.LR)
        btm=Face2x2(self.bottom.UR, self.bottom.LR,self.bottom.UL,self.bottom.LL)
        f=Face2x2(self.face.UL, self.face.UR, self.right.LL, self.right.LR)
        r=Face2x2(self.right.UL, self.right.UR, self.back.LR, self.back.LL)
        bc=Face2x2(self.back.UL, self.back.UR, self.left.LR, self.left.LL)
        l=Face2x2(self.left.UL, self.left.UR, self.face.LL, self.face.LR)
        
        return Cube(f, l, t, r, bc, btm)
    
    def Fcw(self):
        bc=Face2x2(self.back.UL, self.back.UR, self.back.LL, self.back.LR)
        f = Face2x2(self.face.LL, self.face.UL, self.face.LR, self.face.UR)
        t=Face2x2(self.top.UL, self.top.UR, self.left.LR, self.left.UR)
        l=Face2x2(self.left.UL, self.bottom.UL, self.left.LL, self.bottom.UR)
        btm=Face2x2(self.right.LL, self.right.UL, self.bottom.LL, self.bottom.LR)
        r=Face2x2(self.top.LL, self.right.UR, self.top.LR, self.right.LR)
        return Cube(f, l, t, r, bc, btm)
        
    def Fccw(self):
        bc=Face2x2(self.back.UL, self.back.UR, self.back.LL, self.back.LR)
        f = Face2x2(self.face.UR, self.face.LR, self.face.UL,self.face.LL)
        t=Face2x2(self.top.UL, self.top.UR, self.right.UL, self.right.LL)
        r=Face2x2(self.bottom.UR, self.right.UR, self.bottom.UL, self.right.LR)
        btm=Face2x2(self.left.UR, self.left.LR, self.bottom.LL, self.bottom.LR)
        l=Face2x2(self.left.UL, self.top.LR, self.left.LL, self.top.LL)
        return Cube(f, l, t, r, bc, btm)
    
    def Bcw(self):
        f = Face2x2(self.face.UL, self.face.UR, self.face.LL, self.face.LR)
        bc = Face2x2(self.back.UR, self.back.LR, self.back.UL,self.back.LL)
        t=Face2x2(self.right.UR, self.right.LR, self.top.LL, self.top.LR)
        r=Face2x2(self.right.UL, self.bottom.LR, self.right.LL, self.bottom.LL)
        btm=Face2x2(self.bottom.UL, self.bottom.UR, self.left.UL, self.left.LL)
        l=Face2x2(self.top.UR, self.left.UR, self.top.UL, self.left.LR)
        return Cube(f, l, t, r, bc, btm)
    
    def Bccw(self):
        f = Face2x2(self.face.UL, self.face.UR, self.face.LL, self.face.LR)
        bc = Face2x2(self.back.LL, self.back.UL, self.back.LR, self.back.UR)
        t=Face2x2(self.left.LL, self.left.UL, self.top.LL, self.top.LR)
        l=Face2x2(self.bottom.LL, self.left.UR, self.bottom.LR, self.left.LR)
        btm=Face2x2(self.bottom.UL, self.bottom.UR, self.right.LR, self.right.UR)
        r=Face2x2(self.right.UL, self.top.UL, self.right.LL, self.top.UR)
        return Cube(f, l, t, r, bc, btm)
    
    def get_unique_moves(self):
        return [self.rcw, self.rccw, self.tcw, self.tccw, self.Bcw, self.Bccw]

    
    def get_moves(self):
        return [self.lccw, self.lcw, self.rcw,
                self.rccw, self.tcw, self.tccw,
                self.bcw, self.bccw, self.Fcw,
                self.Fccw, self.Bcw, self.Bccw]

    
    def __init__(self, face=face, left=left, top=top, right=right, back=back, bottom=bottom):
        self.left = left
        self.right = right
        self.bottom = bottom
        self.top = top
        self.back = back
        self.face = face
    
    def __repr__(self):
        return str(hash(self))
        #arr = [hash(self.left), hash(self.right), hash(self.bottom), hash(self.top), hash(self.face), hash(self.back)]
        #arr.sort()
        #return str(arr[5]+ 2592*arr[4] + 2592*2592*arr[3])
    
    def  __str__(self):
        return str(self.left)+str(self.face)+str(self.top)+str(self.right)+str(self.back)+str(self.bottom)
    
    def __hash__(self):
        
        #self = self.reorient()
        
        return 1256**2 * hash(self.left)+ 1256*hash(self.top) + hash(self.face) + 1256**3*hash(self.bottom) + 1256**4*hash(self.back)
    def __eq__(self, other):
        try:
            return hash(self) == hash(other)
        except:
            return False
           

## Tests

### Random cube generation

In [3]:
import random

def random_cube(n):
    cube=Cube()
    for i in range(n):
        a = random.randint(0, 11)
        cube=cube.get_moves()[a]()
    return cube

### Visual sanity check for individual rotations

In [4]:

cube= Cube(Face2x2(1, 2, 3, 4),Face2x2(5, 6, 7, 8),Face2x2(9, 10,11, 12),Face2x2(13, 14, 15, 16),Face2x2(17, 18, 19, 20),Face2x2(21, 22, 23, 24))

print(cube)
print()
print(cube.rcw())
print(cube.rccw())
print(cube.lcw())
print(cube.lccw())
print()
print(cube.tccw())
print(cube.tcw())
print(cube.bccw())
print(cube.bcw())
print()
print(cube.Fcw())
print(cube.Fccw())
print(cube.Bcw())
print(cube.Bccw())



[5, 6, 7, 8][1, 2, 3, 4][9, 10, 11, 12][13, 14, 15, 16][17, 18, 19, 20][21, 22, 23, 24]

[5, 6, 7, 8][1, 22, 3, 24][9, 2, 11, 4][15, 13, 16, 14][17, 12, 19, 10][21, 20, 23, 18]
[5, 6, 7, 8][1, 10, 3, 12][9, 20, 11, 18][14, 16, 13, 15][17, 24, 19, 22][21, 2, 23, 4]
[7, 5, 8, 6][9, 2, 11, 4][19, 10, 17, 12][13, 14, 15, 16][23, 18, 21, 20][1, 22, 3, 24]
[6, 8, 5, 7][21, 2, 23, 4][1, 10, 3, 12][13, 14, 15, 16][11, 18, 9, 20][19, 22, 17, 24]

[18, 17, 7, 8][5, 6, 3, 4][10, 12, 9, 11][1, 2, 15, 16][14, 13, 19, 20][21, 22, 23, 24]
[1, 2, 7, 8][13, 14, 3, 4][11, 9, 12, 10][18, 17, 15, 16][6, 5, 19, 20][21, 22, 23, 24]
[5, 6, 3, 4][1, 2, 15, 16][9, 10, 11, 12][13, 14, 20, 19][17, 18, 8, 7][22, 24, 21, 23]
[5, 6, 20, 19][1, 2, 7, 8][9, 10, 11, 12][13, 14, 3, 4][17, 18, 16, 15][23, 21, 24, 22]

[5, 21, 7, 22][3, 1, 4, 2][9, 10, 8, 6][11, 14, 12, 16][17, 18, 19, 20][15, 13, 23, 24]
[5, 12, 7, 11][2, 4, 1, 3][9, 10, 13, 15][22, 14, 21, 16][17, 18, 19, 20][6, 8, 23, 24]
[10, 6, 9, 8][1, 2, 3, 4][14,

### More testing

In [5]:
cube=random_cube(15)

invs=set()
for k in cube.get_invariants():
    invs.update(k.get_invariants())
assert(len(invs)==24)

invs=set(cube.lcw().get_invariants())
invs.update(cube.rcw().get_invariants())
assert(len(invs)==24)

invs=set(cube.Bcw().get_invariants())
invs.update(cube.Fcw().get_invariants())
assert(len(invs)==24)

invs=set(cube.tcw().get_invariants())
invs.update(cube.bcw().get_invariants())
assert(len(invs)==24)

invs=set(cube.tccw().get_invariants())
invs.update(cube.bccw().get_invariants())
assert(len(invs)==24)

invs=set(cube.lccw().get_invariants())
invs.update(cube.rccw().get_invariants())
assert(len(invs)==24)

invs=set(cube.Fccw().get_invariants())
invs.update(cube.Bccw().get_invariants())
assert(len(invs)==24)


assert(cube in cube.lcw().lcw().lcw().lcw().get_invariants())
assert(cube in cube.lccw().lccw().lccw().lccw().get_invariants())
assert(cube in cube.lcw().lccw().get_invariants())
       
assert(cube in cube.rcw().rcw().rcw().rcw().get_invariants())
assert(cube in cube.rcw().rccw().get_invariants())
assert(cube in cube.rccw().rccw().rccw().rccw().get_invariants())


assert(cube in cube.lcw().rccw().get_invariants())

assert(cube in cube.rcw().bccw().rccw().bcw().rcw().bccw().rccw().bcw().rcw().bccw().rccw().bcw().rcw().bccw().rccw().bcw().rcw().bccw().rccw().bcw().rcw().bccw().rccw().bcw().get_invariants())
assert(cube in cube.lcw().tccw().lccw().tcw().lcw().tccw().lccw().tcw().lcw().tccw().lccw().tcw().lcw().tccw().lccw().tcw().lcw().tccw().lccw().tcw().lcw().tccw().lccw().tcw().get_invariants())

assert(cube in cube.tcw().Fccw().tccw().Fcw().tcw().Fccw().tccw().Fcw().tcw().Fccw().tccw().Fcw().tcw().Fccw().tccw().Fcw().tcw().Fccw().tccw().Fcw().tcw().Fccw().tccw().Fcw().get_invariants())
assert(cube in cube.lcw().Fccw().lccw().Fcw().lcw().Fccw().lccw().Fcw().lcw().Fccw().lccw().Fcw().lcw().Fccw().lccw().Fcw().lcw().Fccw().lccw().Fcw().lcw().Fccw().lccw().Fcw().get_invariants())

assert(cube in cube.rcw().Bccw().rccw().Bcw().rcw().Bccw().rccw().Bcw().rcw().Bccw().rccw().Bcw().rcw().Bccw().rccw().Bcw().rcw().Bccw().rccw().Bcw().rcw().Bccw().rccw().Bcw().get_invariants())
assert(cube in cube.Bcw().rccw().Bccw().rcw().Bcw().rccw().Bccw().rcw().Bcw().rccw().Bccw().rcw().Bcw().rccw().Bccw().rcw().Bcw().rccw().Bccw().rcw().Bcw().rccw().Bccw().rcw().get_invariants())

assert(cube in cube.tcw().Bcw().tccw().Bccw().tcw().Bcw().tccw().Bccw().tcw().Bcw().tccw().Bccw().tcw().Bcw().tccw().Bccw().tcw().Bcw().tccw().Bccw().tcw().Bcw().tccw().Bccw().get_invariants())
assert(cube in cube.tcw().rcw().tccw().rccw().tcw().rcw().tccw().rccw().tcw().rcw().tccw().rccw().tcw().rcw().tccw().rccw().tcw().rcw().tccw().rccw().tcw().rcw().tccw().rccw().get_invariants())

assert(cube.lcw().Fcw() not in cube.get_invariants())
assert(cube.lcw().Fccw() not in cube.get_invariants())
assert(cube.lcw().Bcw() not in cube.get_invariants())
assert(cube.lcw().Bccw() not in cube.get_invariants())
assert(cube.lccw().Fcw() not in cube.get_invariants())
assert(cube.lccw().Fccw() not in cube.get_invariants())
assert(cube.lccw().Bcw() not in cube.get_invariants())
assert(cube.lccw().Bccw() not in cube.get_invariants())
assert(cube.lcw().tcw()  not in cube.get_invariants())
assert(cube.lcw().tccw() not in cube.get_invariants())
assert(cube.lcw().bcw() not in cube.get_invariants())
assert(cube.lcw().bccw()  not in cube.get_invariants())
assert(cube.lccw().tcw()  not in cube.get_invariants())
assert(cube.lccw().tccw() not in cube.get_invariants())
assert(cube.lccw().bcw() not in cube.get_invariants())
assert(cube.lccw().bccw() not in cube.get_invariants())

assert(cube not in cube.rcw().Fcw().get_invariants())
assert(cube not in cube.rcw().Fccw().get_invariants())
assert(cube not in cube.rcw().Bcw().get_invariants())
assert(cube not in cube.rcw().Bccw().get_invariants())
assert(cube not in cube.rccw().Fcw().get_invariants())
assert(cube not in cube.rccw().Bccw().get_invariants())
assert(cube not in cube.rccw().Bcw().get_invariants())
assert(cube not in cube.rccw().Bccw().get_invariants())
assert(cube not in cube.rcw().tcw().get_invariants())
assert(cube not in cube.rcw().tccw().get_invariants())
assert(cube not in cube.rcw().bcw().get_invariants())
assert(cube not in cube.rcw().bccw().get_invariants())
assert(cube not in cube.rccw().tcw().get_invariants())
assert(cube not in cube.rccw().tccw().get_invariants())
assert(cube not in cube.rccw().bcw().get_invariants())
assert(cube not in cube.rccw().bccw().get_invariants())

assert(cube.rcw() not in  cube.get_invariants())
assert(cube.rccw() not in cube.get_invariants())
assert(cube.tcw() not in cube.get_invariants())
assert(cube.tccw() not in cube.get_invariants())
assert(cube.Fccw() not in cube.get_invariants())
assert(cube.Fcw() not in cube.get_invariants())



assert(cube.rccw() not in  cube.rcw().get_invariants())
assert(cube.rcw() not in cube.rccw().get_invariants())
assert(cube.tccw()not in  cube.tcw().get_invariants())
assert(cube.tcw() not in   cube.tccw().get_invariants())
assert(cube.Fcw() not in cube.Fccw().get_invariants())
assert(cube.Fccw() not in  cube.Fcw().get_invariants())

assert(cube.rcw() not in cube.rccw().get_invariants())
assert(cube.tccw()not in  cube.rccw().get_invariants())
assert(cube.tcw() not in   cube.rccw().get_invariants())
assert(cube.Fcw() not in cube.rccw().get_invariants())
assert(cube.Fccw() not in   cube.rccw().get_invariants())

assert(cube.tccw()not in  cube.rcw().get_invariants())
assert(cube.tcw() not in   cube.rcw().get_invariants())
assert(cube.Fcw() not in cube.rcw().get_invariants())
assert(cube.Fccw() not in   cube.rcw().get_invariants())

assert(cube.tcw() not in   cube.tccw().get_invariants())
assert(cube.Fcw() not in cube.tccw().get_invariants())
assert(cube.Fccw() not in   cube.tccw().get_invariants())

assert(cube.Fcw() not in cube.tcw().get_invariants())
assert(cube.Fccw() not in   cube.tcw().get_invariants())

assert(cube in cube.lcw().rccw().get_invariants())


assert(cube in cube.tcw().tcw().tcw().tcw().get_invariants())
assert(cube in cube.tcw().tccw().get_invariants())
assert(cube in cube.tccw().tccw().tccw().tccw().get_invariants())

assert(cube in cube.bcw().bcw().bcw().bcw().get_invariants())
assert(cube in cube.bcw().bccw().get_invariants())
assert(cube in cube.bccw().bccw().bccw().bccw().get_invariants())

assert(cube in cube.Bcw().Bcw().Bcw().Bcw().get_invariants())
assert(cube in cube.Bcw().Bccw().get_invariants())
assert(cube in cube.Bccw().Bccw().Bccw().Bccw().get_invariants())

assert(cube in cube.Fcw().Fcw().Fcw().Fcw().get_invariants())
assert(cube in cube.Fcw().Fccw().get_invariants())
assert(cube in cube.Fccw().Fccw().Fccw().Fccw().get_invariants())

## Running the BFS

In [ ]:
import time
cube=Cube()
all_states = set()
all_states.update(cube.get_invariants())
next_layer = set()
current_layer = set()
current_layer.add(cube)
ctr=1
while True:
    start = time.process_time()
    for cube in current_layer:
        for move in cube.get_moves():
            new_cube = move()
            new_cube_hash = hash(new_cube)

            if new_cube_hash not in all_states:
                invs = new_cube.get_invariants()
                all_states.update(invs)
                next_layer.add(new_cube)
            
    confs=len(all_states)
    if len(next_layer)==0:
        break
    current_layer=next_layer.copy()
    print(ctr,len(next_layer), time.process_time() - start)
    next_layer=set()
    ctr+=1
 

1 6 0.015625
2 27 0.03125
3 120 0.09375
4 534 0.484375
5 2256 2.015625
6 8969 8.9375
7 33058 36.09375
8 114149 128.4375
9 360508 578.828125


## Results
Proposed algorithm found configurations up to depth 9. After that the memory became the limiting factor and noteboke froze. The memory consumption could be reduced by keeping only one orientation of the cube. My implementation held all 24 invariants, because calculating them on the fly for each comparison was too expensive compute-wise. The computation could be done more efficently if 24 invariants could be found by an effienct functions(I used regular moves to reach those invariant cubes).
